# Imports

In [271]:
import pandas as pd
import numpy as np
import random

In [3]:
data_DB = pd.read_csv("Food_DB.csv")

In [8]:
# Subset of list
data_dinner = data_DB[pd.isnull(data_DB["Lunch"])]
data_lunch = data_DB[data_DB["Lunch"] == "x"]

In [9]:
data_lunch.head()

,Meal,Amount,Stars (1-5),Effort (1-5),Lunch,Notes
4,Hotdog,2,5,1,x,NaN
15,Nuggies & Fries,2,4,1,x,NaN
16,Fish & Chips,2,4,2,x,NaN
31,Nudlar,2,3,3,x,NaN
32,Plättar,2,5,3,x,NaN


# Main functions

In [522]:
# SETTINGS
days_in_plan = 7
starting_meal = 1
current_pos = [starting_meal, 1]
meals_left = 0
current_meal = [None]
leftover_meal = None
leftover_amount = 0
chosen_meals = []
single_lunch_days = [5]
non_ready_lunch_days = [3,4]

In [514]:
# Create probability
data_DB["Probability"] = ((data_DB["Stars (1-5)"] / 5) * 0.75) + ((1 - 0.2 * (data_DB["Effort (1-5)"] - 1)) * 0.25)

In [515]:
# Create mealplan_DB
mealplan_DB = pd.DataFrame()
mealplan_DB["meal_of_day"] = ["lunch", "dinner"]
days_in_plan_range = range(0,days_in_plan)
for i in days_in_plan_range:
    mealplan_DB[i + 1] = 0

In [516]:
mealplan_DB.head()

,meal_of_day,1,2,3,4,5,6,7
0,lunch,0,0,0,0,0,0,0
1,dinner,0,0,0,0,0,0,0


In [517]:
def pick_random_meal(meal_db, current_meal, chosen_meals=None, min_amount=0):
    # Filter meals based on the amount criteria and exclude chosen meals
    eligible_meals = meal_db[(meal_db['Amount'] >= min_amount) & (~meal_db['Meal'].isin(chosen_meals))]

    if not eligible_meals.empty:
        # Pick a random meal from eligible meals
        random_row = eligible_meals.sample()
        random_meal = random_row.iloc[0]

        # Update the current meal variable
        current_meal[0] = random_meal['Meal']

        # Add the chosen meal to the list of chosen meals
        chosen_meals.append(current_meal[0])

        return random_meal
    else:
        return "No eligible meals available."

In [512]:
pick_random_meal(data_dinner, current_meal, chosen_meals, min_amount=4)

'No eligible meals available.'

In [519]:
def generate_meal_plan(num_days, data_dinner, data_lunch, single_lunch_days, non_ready_lunch_days):
    # Initialize the meal plan matrix
    meal_plan = np.zeros((2, num_days), dtype=int)

    # Loop through each day
    for day in range(num_days):
        # Dinner
        dinner_choice = pick_random_meal(data_dinner, meal_plan[1, day], chosen_meals=None)
        meal_plan[1, day] = dinner_choice

        # Lunch
        lunch_choice = 0  # Default: Ready-made leftovers

        if day + 1 in single_lunch_days:
            # Single-person lunch day
            lunch_choice = pick_random_meal(data_lunch, meal_plan[0, day], chosen_meals=None)

        elif day + 1 in non_ready_lunch_days:
            # Non-ready-made lunch day
            lunch_choice = pick_random_meal(data_lunch, meal_plan[0, day], chosen_meals=None, min_amount=1)

        meal_plan[0, day] = lunch_choice

    return meal_plan

In [523]:
# Example usage:
num_days = 7
single_lunch_days = []
ready_lunch_days = [3,4,5,6,7]

meal_plan_result = generate_meal_plan(num_days, data_dinner, data_lunch, single_lunch_days, non_ready_lunch_days)
print(meal_plan_result)

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [NoneType]